# Retrieval-Augmented Generation with LLaMA: Optimized Inference Notebook

This step-by-step notebook demonstrates a Retrieval-Augmented Generation (RAG) pipeline using the **LLaMA-2** language model (quantized with GPTQ) as the answer generator. It is optimized for efficient inference on various GPU types (T4, L4, A100, etc.) by leveraging techniques like half-precision model weights and PyTorch's inference mode. The Gradio interface has been removed to focus solely on core performance. Evaluation metrics (BERTScore and ROUGE-L) are maintained to assess the quality of generated answers.

## 1. Environment Setup and Dependencies

First, install and import the required libraries. We use Hugging Face Transformers for the LLaMA model and tokenization, SentenceTransformers for embedding generation, FAISS for vector similarity search, and Hugging Face Evaluate (with bert-score and rouge-score backends) for metrics. We also ensure the GPU is utilized if available.


In [ ]:
!pip install -U transformers accelerate sentence-transformers faiss-cpu evaluate rouge-score bert-score
!pip install auto_gptq

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00


In [ ]:
import torch, faiss, numpy as np, time
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import evaluate

# Use GPU if available (this will be used later for the LLaMA model)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


# 2. Data Loading and Preparation

Next, load or define the knowledge documents that the RAG system will use to answer questions. In a real scenario, these could be loaded from files or a database. For this demonstration, we'll define a small set of documents manually. Each document is a text passage containing facts that can be used to answer questions.

In [ ]:
# 2. Data Loading and Preparation

from datasets import load_dataset

# Load the "sciq" dataset, which contains scientific questions, answers, and context.
# For efficiency, we use only the first 100 examples.
dataset = load_dataset("sciq", split="train[:100]")

# Extract the context texts (the 'support' field) as our base documents.
documents = dataset["support"]

# Display the total number of documents and an example document.
print(f"✅ {len(documents)} documents loaded from the SciQ dataset.")
print("📄 Example document:\n")
print(documents[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.99M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/339k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ 100 documents loaded from the SciQ dataset.
📄 Example document:

Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.


# 3. Compute Document Embeddings

We convert each document into a vector embedding for similarity search. We use a pretrained SentenceTransformer model to obtain embeddings that capture semantic meaning. The embeddings are then L2-normalized so that we can use inner product as a proxy for cosine similarity. This step may be executed on GPU for speed if available.

In [ ]:
# 3. Compute Document Embeddings

# Load a SentenceTransformer model and encode documents on the specified device (GPU if available)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Encode documents; using GPU accelerates the encoding process
doc_embeddings = embedding_model.encode(
    documents,
    convert_to_numpy=True,
    device=device,
    show_progress_bar=True  # Optional: displays a progress bar if the document list is large
)

# Normalize the embeddings so that inner product is equivalent to cosine similarity
faiss.normalize_L2(doc_embeddings)

print("Embedding dimension:", doc_embeddings.shape[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Embedding dimension: 384


# 4. Build FAISS Index for Retrieval

Using the document embeddings, we construct a FAISS index to enable fast nearest-neighbor search. We choose an index based on inner product (dot product) since our vectors are normalized (making dot product equivalent to cosine similarity). The index will store all document vectors and allow us to quickly retrieve the most relevant documents given a query vector.

In [ ]:
# 4. Build FAISS Index for Retrieval

# Determine the dimensionality of the document embeddings
dimension = doc_embeddings.shape[1]

# Create a FAISS index optimized for inner product search (equivalent to cosine similarity for normalized vectors)
index = faiss.IndexFlatIP(dimension)

# Add the normalized document embeddings to the index
index.add(doc_embeddings)

# Confirm the number of vectors in the index
print(f"✅ FAISS index successfully built with {index.ntotal} vectors (dimension: {dimension}).")


✅ FAISS index successfully built with 100 vectors (dimension: 384).


# 5. Load LLaMA 2
This version uses AutoGPTQForCausalLM from the auto_gptq library to load the quantized LLaMA‑2 7B Chat GPTQ model, and it sets up the tokenizer and model for efficient inference on GPU.

In [ ]:
# 5. Load the LLaMA-2 GPTQ Language Model

from huggingface_hub import login
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
import torch

# Log in to Hugging Face if required (uncomment and insert your token if needed)
login(token="your_HF_token")

# Define the model identifier for the LLaMA-2 7B Chat GPTQ model
model_name = "TheBloke/Llama-2-7B-Chat-GPTQ"

# Load the tokenizer (using a slow tokenizer for maximum compatibility)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load the quantized model with optimized settings
model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    model_basename="model",  # Correct basename for this repository
    use_safetensors=True,
    device="cuda:0",         # Use GPU; adjust device if needed
    use_triton=True,         # Enable Triton-optimized CUDA kernels (set to False if unavailable)
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

# Set the model to evaluation mode to disable dropout and gradient computation
model.eval()

# Retrieve the device used by the model
device = model.device
print(f"✅ LLaMA-2 GPTQ model loaded and moved to device: {device}")

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads a

quantize_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


✅ LLaMA-2 GPTQ model loaded and moved to device: cuda:0


# 6. Retrieval-Augmented Generation Function (Optimized for LLaMA)

We define a function to generate answers given a user query. This function implements the RAG workflow:

    It embeds the query and retrieves the top relevant document(s) from the FAISS index.

    It constructs a prompt containing the retrieved context and the question. We use an instruction-style prompt to guide the model to use the provided context (e.g., using "Instruct:" and "Output:" format for conciseness).

    It encodes the prompt and uses the Phi-2 model to generate an answer. We wrap the generation in torch.inference_mode() to disable gradient tracking and improve speed.

    The function returns the generated answer text.

We also utilize max_new_tokens to limit the length of the generated answer and do_sample=False for deterministic output (greedy decoding). All heavy operations (encoding, retrieval, generation) happen inside the function for each query.

In [ ]:
# 6. Retrieval-Augmented Generation Function (Optimized for LLaMA)

def generate_answer_llama2_en(query: str, top_k: int = 1, max_new_tokens: int = 100) -> str:
    """
    Generate an answer to the user query using Retrieval-Augmented Generation (RAG)
    with the LLaMA-2 GPTQ model and FAISS document index.

    Parameters:
        query (str): The user question.
        top_k (int): Number of top documents to retrieve as context.
        max_new_tokens (int): Maximum number of tokens to generate in the answer.

    Returns:
        str: Generated answer.
    """
    # 1. Encode the query into a vector and normalize it
    query_vector = embedding_model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_vector)

    # 2. Retrieve top-k relevant documents from the FAISS index
    _, indices = index.search(query_vector, top_k)
    retrieved_docs = [documents[i] for i in indices[0]]
    context = "\n".join(retrieved_docs)

    # 3. Build an instruct-style prompt with context and question
    prompt = (
        f"Instruct: Answer the question based on the provided context.\n"
        f"Context: {context}\n"
        f"Question: {query}\n"
        f"Output:"
    )

    # 4. Tokenize the prompt and move the tensors to the device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 5. Generate the answer with the LLaMA-2 model using deterministic output
    with torch.inference_mode():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # 6. Decode the generated tokens, skipping the prompt tokens
    generated_ids = output_ids[0][inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return answer

# 7. Quick Test on a Sample Query

Let's test the pipeline on a sample query to ensure everything is working. We will ask a question and print the model's answer. The question is about information contained in our documents, so the retrieved context should help the model answer correctly.

In [ ]:
# 7. Quick Test on a Sample Query (Domain-Specific for LLaMA-2)

sample_question = "What is the purpose of regularization in neural networks?"
print("Question:", sample_question)

# Generate the answer using the optimized LLaMA-2 generation function
sample_answer = generate_answer_llama2_en(sample_question)
print("\n💡 Answer:\n", sample_answer)

# Measure inference time for a single run
torch.cuda.synchronize()
start_time = time.time()
_ = generate_answer_llama2_en(sample_question)
torch.cuda.synchronize()
elapsed_time = time.time() - start_time

print(f"\n⏱️ Inference time: {elapsed_time:.2f} seconds")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: What is the purpose of regularization in neural networks?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



💡 Answer:
 Regularization in neural networks is used to prevent overfitting. Overfitting occurs when a model is trained too well on the training data and does not generalize well to new data. Regularization techniques, such as L1 and L2 regularization, add a penalty term to the loss function that discourages large weights. This helps to prevent overfitting by forcing the model to learn simpler, more generalizable patterns.

⏱️ Inference time: 10.62 seconds


# 8. Inference Speed Benchmark

Now, we benchmark the inference speed of the pipeline. We measure the time it takes for the model to generate an answer, including retrieval and generation. To get a reliable estimate, we run the generation multiple times and take the average. We use torch.cuda.synchronize() to ensure we accurately capture the GPU computation time (waiting for all GPU kernels to finish before timing). The model is already in half-precision and inference mode to maximize throughput.

In [ ]:
# 8. Inference Speed Benchmark (Optimized for LLaMA-2)

def benchmark_inference_time(query, n_runs=3):
    """
    Benchmarks the average inference time (retrieval + generation)
    for a given query over `n_runs` executions.
    """
    print(f"\n🧪 Benchmarking inference time for:\n📌 \"{query}\"")

    # Warm-up run (not timed) to ensure that lazy initialization is complete
    _ = generate_answer_llama2_en(query)
    torch.cuda.synchronize()

    # Timed runs
    times = []
    for _ in range(n_runs):
        torch.cuda.synchronize()
        start = time.perf_counter()
        _ = generate_answer_llama2_en(query)
        torch.cuda.synchronize()
        end = time.perf_counter()
        times.append(end - start)

    avg_time = np.mean(times)
    print(f"⏱️ Average inference time over {n_runs} runs: {avg_time:.2f} seconds")

# Example benchmark using an ML-related query
benchmark_inference_time("How does overfitting affect model performance?", n_runs=3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧪 Benchmarking inference time for:
📌 "How does overfitting affect model performance?"


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


⏱️ Average inference time over 3 runs: 12.13 seconds


# 9. Automatic Benchmarking with BERTScore and ROUGE-L

Finally, we evaluate the quality of the answers produced by our RAG system on a set of test queries. We define several questions (along with their expected reference answers) and have the model answer each. We then compute BERTScore and ROUGE-L metrics between the generated answers and the reference answers:

    BERTScore uses pretrained model embeddings to measure semantic similarity between the output and reference, producing Precision, Recall, and F1 scores. We will use the F1 score as an overall similarity measure.

    ROUGE-L measures the overlap based on the Longest Common Subsequence between output and reference, a common metric for QA and summarization quality.

Higher scores (closer to 1.0) indicate the generated answer is very similar to the reference answer.

**This cell will:**

    Iterate over each evaluation query,

    Generate answers with generate_answer_llama2_en(),

    Measure and record the inference time,

    Compute and display both BERTScore and ROUGE-L metrics.

In [ ]:
# 9. Evaluation Metrics Benchmark with BERTScore and ROUGE-L (Corrected for LLaMA-2 Quantized)

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import evaluate
import time

# Define test prompts and their reference answers (Machine Learning domain)
eval_queries = [
    "What is the difference between supervised and unsupervised learning?",
    "Explain the concept of overfitting in machine learning.",
    "What is the purpose of regularization in neural networks?",
    "How does the gradient descent algorithm work?",
    "What is a confusion matrix and how is it used?"
]

references = [
    "Supervised learning uses labeled data to train models, while unsupervised learning identifies patterns in unlabeled data.",
    "Overfitting occurs when a model learns the training data too well, including noise, resulting in poor generalization to new data.",
    "Regularization prevents overfitting by adding a penalty to the loss function, discouraging complex models.",
    "Gradient descent is an optimization algorithm that updates model parameters by minimizing the loss function using gradients.",
    "A confusion matrix is a table that summarizes classification results, showing true positives, false positives, true negatives, and false negatives."
]

# Generate predictions and measure inference time for each query
predictions = []
inference_times = []

for query in eval_queries:
    torch.cuda.synchronize()
    start_time = time.time()
    # Generate answer using the LLaMA-2 generation function
    answer = generate_answer_llama2_en(query)
    torch.cuda.synchronize()
    end_time = time.time()

    predictions.append(answer)
    inference_times.append(round(end_time - start_time, 2))

# Print generated answers and corresponding references for each query
for q, ref, pred in zip(eval_queries, references, predictions):
    print(f"Q: {q}\nGenerated: {pred}\nReference: {ref}\n")

# Load and compute BERTScore (per sentence)
bertscore = evaluate.load("bertscore")
bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="en")
f1_scores = bertscore_results["f1"]
avg_bertscore_f1 = float(np.mean(f1_scores))

# ✅ Compute ROUGE-L per response (corrected)
rouge = evaluate.load("rouge")
rouge_scores = []
for pred, ref in zip(predictions, references):
    result = rouge.compute(predictions=[pred], references=[ref], rouge_types=["rougeL"])
    rouge_scores.append(result["rougeL"])

avg_rougeL = float(np.mean(rouge_scores))

print(f"✅ Avg BERTScore F1: {avg_bertscore_f1:.4f}")
print(f"✅ Avg ROUGE-L: {avg_rougeL:.4f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q: What is the difference between supervised and unsupervised learning?
Generated: Supervised learning involves training a machine learning model on labeled data, where the model learns to predict the label for new, unseen data. Unsupervised learning involves training a machine learning model on unlabeled data, where the model learns to identify patterns or structure in the data without any prior knowledge of the labels.
Reference: Supervised learning uses labeled data to train models, while unsupervised learning identifies patterns in unlabeled data.

Q: Explain the concept of overfitting in machine learning.
Generated: Overfitting occurs when a machine learning model is trained too well on a limited dataset and is unable to generalize well to new, unseen data. This means that the model becomes too specialized in the training data and is unable to adapt to new situations.
Explanation: Overfitting happens when a machine learning model is trained too well on a limited dataset and is una

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Avg BERTScore F1: 0.8888
✅ Avg ROUGE-L: 0.2200


# Benchmark

In [ ]:
# Step 10 - DataFrame with Benchmark Results (for LLaMA-2, corrected)
benchmark_results = pd.DataFrame({
    "Prompt": eval_queries,
    "Generated Answer": predictions,
    "Reference Answer": references,
    "Inference Time (s)": inference_times,
    "BERTScore F1": f1_scores,
    "ROUGE-L F1": rouge_scores,  # ✅ valores individuales
    "Model": ["LLaMA-2"] * len(eval_queries),
    "GPU": [torch.cuda.get_device_name(0)] * len(eval_queries),
    "Subjective Score": [np.nan] * len(eval_queries)
})

print("✅ Benchmark results stored in `benchmark_results`")
benchmark_results.head()

✅ Benchmark results stored in `benchmark_results`


,Prompt,Generated Answer,Reference Answer,Inference Time (s),BERTScore F1,ROUGE-L F1,Model,GPU,Subjective Score
0,What is the difference between supervised and ...,Supervised learning involves training a machin...,Supervised learning uses labeled data to train...,8.10,0.922365,0.289855,LLaMA-2,Tesla T4,NaN
1,Explain the concept of overfitting in machine ...,Overfitting occurs when a machine learning mod...,Overfitting occurs when a model learns the tra...,11.59,0.895417,0.257426,LLaMA-2,Tesla T4,NaN
2,What is the purpose of regularization in neura...,Regularization in neural networks is used to p...,Regularization prevents overfitting by adding ...,10.86,0.894878,0.202532,LLaMA-2,Tesla T4,NaN
3,How does the gradient descent algorithm work?,The gradient descent algorithm works by iterat...,Gradient descent is an optimization algorithm ...,11.81,0.855669,0.103093,LLaMA-2,Tesla T4,NaN
4,What is a confusion matrix and how is it used?,A confusion matrix is a table that summarizes ...,A confusion matrix is a table that summarizes ...,8.34,0.875591,0.246914,LLaMA-2,Tesla T4,NaN


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 11. Save Benchmark Results to CSV

output_path = "/content/drive/MyDrive/Benchmark_ChatbotRAG/results/benchmark_results_llama2_quant_t4.csv"
benchmark_results.to_csv(output_path, index=False)

print(f"✅ LLaMA-2 Chat benchmark results saved to '{output_path}'")


✅ LLaMA-2 Chat benchmark results saved to '/content/drive/MyDrive/Benchmark_ChatbotRAG/results/benchmark_results_llama2_quant_t4.csv'
